In [1]:
!pip install pymdptoolbox
import mdptoolbox.example
import numpy as np
import pandas as pd
import itertools
from collections import defaultdict
from collections import Counter
import random
import timeit
import random
import time

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
np.random.seed(123)


datap = pd.read_csv('train.csv', sep=',', header=0)

states=14
actions=14
p = [1 for a in range(0,states)]
a=list(range(1, actions+1))
s=list(range(1, states+1))


#get transition probabilities

sas=pd.DataFrame({'count' : datap.groupby( [ 's', 'a','sp'] ).size()}).reset_index()
sa=pd.DataFrame({'count' : datap.groupby( [ 's', 'a'] ).size()}).reset_index()

t = pd.merge(sas, sa,  how='left', left_on=['s','a'], right_on = ['s','a'])
t['t_prob'] = t['count_x']/t['count_y']

r=datap[['s','a','r']].groupby([ 's', 'a']).agg({'count', 'sum'}).reset_index()

#print(r)

model= pd.merge(t, r,  how='left', left_on=['s','a'], right_on = ['s','a'])

r.columns = ['s', 'a','r_sum','count']

model.columns = ['s', 'a','sp','count_sas','count_sa','t_prob','r_count','r_sum']
#print(model)


model['r_val'] = model['r_sum']/model['count_sa']
model['r_expected'] = model['t_prob']*model['r_val']
small_model= model.loc[(model['a']== 1)]

#print(small_model)

transition=np.zeros((14,14))
transition[0][1]=1
#print(transition)
for index, row in small_model.iterrows():
    transition[int(row['s']-1)][int(row['sp']-1)]=row['t_prob']

#print(transition)
#print(small_model)

#get expected avg R
r_s=datap[['s','a','sp','r']].groupby([ 's', 'a','sp']).agg({'count', 'sum'}).reset_index()
r_s.columns = ['s', 'a','sp','r_count','r_sum']
r_s['avg_r']=r_s['r_sum']/r_s['r_count']

initial_s=model[['sp','count_sas']].groupby([ 'sp']).agg({'sum'}).reset_index()
initial_s.columns = ['sp', 'count']
initial_s['perc']=initial_s['count']/initial_s['count'].sum()
#print(r_s)


C:\Users\fxi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\fxi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [6]:
def simulate(policy, steps):
    #initial state 
    if steps <1:
        return
        
    step=0
    state=np.random.choice(np.arange(1, 15), p=initial_s.perc.values)
    #print("Step:",step, "State:",state)
    
    while step<steps:
        step=step+1
        action=policy[state-1]
        prev_state=state
        state=np.random.choice(np.arange(1, 15), p=transition[state-1])
        reward=r_s.loc[(r_s['s']== prev_state)& (r_s['a']== action)& (r_s['sp']== state)]['avg_r'].mean()
        print("Step:",step, "S:",prev_state,"A:",action,"Sp:",state, "R:",reward)
        
sample_policy=[13,  2,  7,  5, 13, 13, 13,  8,  3,  5, 11,  3, 13,  8]
simulate(sample_policy,10)

Step: 1 S: 8 A: 8 Sp: 7 R: -0.3333333333333333
Step: 2 S: 7 A: 13 Sp: 1 R: -0.3333333333333333
Step: 3 S: 1 A: 13 Sp: 5 R: -0.5
Step: 4 S: 5 A: 13 Sp: 8 R: -0.3333333333333333
Step: 5 S: 8 A: 8 Sp: 5 R: -0.3333333333333333
Step: 6 S: 5 A: 13 Sp: 3 R: -0.3333333333333333
Step: 7 S: 3 A: 7 Sp: 5 R: -0.3333333333333333
Step: 8 S: 5 A: 13 Sp: 7 R: -0.3333333333333333
Step: 9 S: 7 A: 13 Sp: 7 R: -0.3333333333333333
Step: 10 S: 7 A: 13 Sp: 11 R: -0.3333333333333333
